# Pandemic Economic Impact Analysis
## Archetype 4: The Economist

This notebook explores the economic impact of pandemics by analyzing real-world data on GDP, unemployment, and sector-specific performance. Students will learn to model supply and demand shocks, analyze labor market disruptions, and evaluate the costs and benefits of public health interventions.

### Learning Objectives:
- Understand the economic mechanisms through which pandemics affect economies
- Analyze macroeconomic indicators during health crises
- Model supply-side and demand-side economic shocks
- Evaluate the economic costs and benefits of public health interventions
- Examine how pandemics exacerbate economic inequality
- Design economic support and stimulus policies

### Key Concepts:
- **Supply shock**: Disruption to production capacity (e.g., workforce illness, supply chain interruption)
- **Demand shock**: Changes in consumer spending patterns (e.g., reduced travel, increased healthcare spending)
- **Labor market effects**: Unemployment, remote work, essential worker dynamics
- **Sectoral heterogeneity**: Different impacts across industries (tourism vs. healthcare vs. technology)
- **Fiscal response**: Government spending and support programs
- **Economic inequality**: Differential impacts across income groups, demographics

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# For economic modeling
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

print("💰 Libraries loaded successfully for economic impact analysis")

## Part 1: Understanding Pandemic Economic Impacts

Pandemics affect economies through multiple channels:

### Supply-Side Impacts:
- **Labor supply reduction**: Workers becoming ill, quarantining, or caring for others
- **Supply chain disruptions**: Border closures, transportation restrictions
- **Business closures**: Mandatory shutdowns of non-essential businesses
- **Reduced investment**: Uncertainty leading to postponed capital projects

### Demand-Side Impacts:
- **Consumption changes**: Reduced spending on services (travel, dining), increased spending on goods (food, technology)
- **Precautionary saving**: Households saving more due to uncertainty
- **Sector reallocation**: Shift from services to goods, from public to private consumption

### Policy Responses:
- **Fiscal policy**: Government spending, tax relief, direct transfers
- **Monetary policy**: Interest rate changes, quantitative easing
- **Health policy**: Public health measures with economic costs and benefits

In [ ]:
# Generate synthetic economic data simulating a pandemic impact
# This represents what you might see in real economic statistics during a health crisis

def generate_pandemic_economic_data():
    """
    Generate synthetic economic data showing pandemic impact
    """
    np.random.seed(42)  # For reproducibility
    
    # Time series: 2019-2023 (monthly data)
    start_date = datetime(2019, 1, 1)
    end_date = datetime(2023, 12, 31)
    dates = pd.date_range(start_date, end_date, freq='MS')  # Month start
    
    # Define pandemic phases
    pandemic_start = datetime(2020, 3, 1)
    lockdown_end = datetime(2020, 6, 1)
    second_wave = datetime(2020, 11, 1)
    recovery_start = datetime(2021, 6, 1)
    
    data = []
    
    # Baseline economic parameters (pre-pandemic)
    baseline_gdp_growth = 0.002  # 2% annual = ~0.2% monthly
    baseline_unemployment = 3.5
    baseline_inflation = 2.0
    
    for i, date in enumerate(dates):
        # Determine pandemic phase
        if date < pandemic_start:
            phase = 'pre_pandemic'
            gdp_shock = 0
            unemployment_shock = 0
            policy_support = 0
        elif date < lockdown_end:
            phase = 'acute_phase'
            gdp_shock = -0.15  # -15% shock
            unemployment_shock = 8.0
            policy_support = 0.05  # 5% GDP stimulus
        elif date < second_wave:
            phase = 'partial_recovery'
            gdp_shock = -0.08
            unemployment_shock = 4.0
            policy_support = 0.03
        elif date < recovery_start:
            phase = 'second_wave'
            gdp_shock = -0.12
            unemployment_shock = 6.0
            policy_support = 0.04
        else:
            phase = 'recovery'
            months_since_recovery = (date - recovery_start).days / 30
            gdp_shock = max(-0.02, -0.08 + 0.01 * months_since_recovery)
            unemployment_shock = max(0.5, 6.0 - 0.2 * months_since_recovery)
            policy_support = max(0, 0.04 - 0.001 * months_since_recovery)
        
        # Add some randomness
        gdp_noise = np.random.normal(0, 0.01)
        unemployment_noise = np.random.normal(0, 0.3)
        
        # Calculate economic indicators
        gdp_growth = baseline_gdp_growth + gdp_shock + gdp_noise + policy_support/12
        unemployment_rate = baseline_unemployment + unemployment_shock + unemployment_noise
        
        # Consumer price index (inflation effects)
        if phase in ['acute_phase', 'second_wave']:
            inflation_shock = np.random.normal(0.5, 0.2)  # Supply chain disruptions
        else:
            inflation_shock = np.random.normal(0, 0.1)
        
        inflation_rate = baseline_inflation + inflation_shock
        
        # Sectoral data
        if phase == 'acute_phase':
            services_impact = -0.35  # Services hit hardest
            manufacturing_impact = -0.20
            agriculture_impact = -0.05
            technology_impact = 0.10  # Technology benefits
        elif phase == 'partial_recovery':
            services_impact = -0.15
            manufacturing_impact = -0.08
            agriculture_impact = 0.02
            technology_impact = 0.15
        elif phase == 'second_wave':
            services_impact = -0.25
            manufacturing_impact = -0.12
            agriculture_impact = -0.02
            technology_impact = 0.08
        elif phase == 'recovery':
            months_since = (date - recovery_start).days / 30
            services_impact = max(-0.05, -0.15 + 0.01 * months_since)
            manufacturing_impact = max(-0.02, -0.08 + 0.008 * months_since)
            agriculture_impact = min(0.05, 0.02 + 0.001 * months_since)
            technology_impact = min(0.20, 0.15 + 0.005 * months_since)
        else:  # pre_pandemic
            services_impact = np.random.normal(0, 0.01)
            manufacturing_impact = np.random.normal(0, 0.015)
            agriculture_impact = np.random.normal(0, 0.02)
            technology_impact = np.random.normal(0.05, 0.02)  # Tech growing pre-pandemic
        
        # Stock market (more volatile)
        if phase == 'acute_phase':
            stock_volatility = 0.15
            stock_trend = -0.05
        elif phase == 'recovery':
            stock_volatility = 0.08
            stock_trend = 0.03
        else:
            stock_volatility = 0.05
            stock_trend = 0.01
        
        stock_return = stock_trend + np.random.normal(0, stock_volatility)
        
        # Government debt (increases with spending)
        debt_increase = policy_support * 1.2  # Multiplier effect
        
        data.append({
            'date': date,
            'phase': phase,
            'gdp_growth_rate': gdp_growth * 12,  # Annualized
            'unemployment_rate': max(0, unemployment_rate),
            'inflation_rate': inflation_rate,
            'services_growth': services_impact * 12,
            'manufacturing_growth': manufacturing_impact * 12,
            'agriculture_growth': agriculture_impact * 12,
            'technology_growth': technology_impact * 12,
            'stock_market_return': stock_return * 12,
            'policy_support_pct_gdp': policy_support,
            'debt_increase': debt_increase,
            'consumer_confidence': max(0, 100 + gdp_shock * 200 + np.random.normal(0, 5))
        })
    
    return pd.DataFrame(data)

# Generate the economic dataset
econ_data = generate_pandemic_economic_data()

# Add cumulative calculations
econ_data['cumulative_gdp_impact'] = ((1 + econ_data['gdp_growth_rate']/100).cumprod() - 1) * 100
econ_data['government_debt_level'] = econ_data['debt_increase'].cumsum() * 100  # As % of GDP

print(f"📊 Generated economic dataset: {len(econ_data)} monthly observations")
print(f"📅 Time period: {econ_data['date'].min().date()} to {econ_data['date'].max().date()}")
print(f"\n🏷️ Pandemic phases:")
phase_counts = econ_data['phase'].value_counts()
for phase, count in phase_counts.items():
    print(f"  {phase.replace('_', ' ').title()}: {count} months")

# Display sample data
print(f"\n📋 Sample Data:")
print(econ_data[['date', 'phase', 'gdp_growth_rate', 'unemployment_rate', 'inflation_rate']].head(10).to_string(index=False))

## Part 2: Macroeconomic Impact Analysis

In [ ]:
# Create comprehensive macroeconomic analysis
fig, axes = plt.subplots(3, 2, figsize=(16, 18))

# Color coding for phases
phase_colors = {
    'pre_pandemic': 'blue',
    'acute_phase': 'red', 
    'partial_recovery': 'orange',
    'second_wave': 'darkred',
    'recovery': 'green'
}

# Plot 1: GDP Growth Rate
for phase in phase_colors.keys():
    phase_data = econ_data[econ_data['phase'] == phase]
    if not phase_data.empty:
        axes[0,0].plot(phase_data['date'], phase_data['gdp_growth_rate'], 
                       color=phase_colors[phase], linewidth=2, 
                       label=phase.replace('_', ' ').title())

axes[0,0].axhline(y=0, color='black', linestyle='--', alpha=0.7)
axes[0,0].set_ylabel('GDP Growth Rate (%)')
axes[0,0].set_title('GDP Growth Rate Over Time')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Unemployment Rate
axes[0,1].plot(econ_data['date'], econ_data['unemployment_rate'], 
               color='red', linewidth=2)
axes[0,1].set_ylabel('Unemployment Rate (%)')
axes[0,1].set_title('Unemployment Rate Over Time')
axes[0,1].grid(True, alpha=0.3)

# Add recession shading
recession_start = datetime(2020, 3, 1)
recession_end = datetime(2021, 6, 1)
axes[0,1].axvspan(recession_start, recession_end, alpha=0.2, color='red', label='Pandemic Period')
axes[0,1].legend()

# Plot 3: Inflation Rate
axes[1,0].plot(econ_data['date'], econ_data['inflation_rate'], 
               color='purple', linewidth=2)
axes[1,0].axhline(y=2.0, color='black', linestyle='--', alpha=0.7, label='Target (2%)')
axes[1,0].set_ylabel('Inflation Rate (%)')
axes[1,0].set_title('Inflation Rate Over Time')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Consumer Confidence
axes[1,1].plot(econ_data['date'], econ_data['consumer_confidence'], 
               color='brown', linewidth=2)
axes[1,1].set_ylabel('Consumer Confidence Index')
axes[1,1].set_title('Consumer Confidence Over Time')
axes[1,1].grid(True, alpha=0.3)

# Plot 5: Cumulative GDP Impact
axes[2,0].plot(econ_data['date'], econ_data['cumulative_gdp_impact'], 
               color='darkblue', linewidth=3)
axes[2,0].axhline(y=0, color='black', linestyle='--', alpha=0.7)
axes[2,0].set_ylabel('Cumulative GDP Impact (%)')
axes[2,0].set_title('Cumulative Economic Impact')
axes[2,0].grid(True, alpha=0.3)

# Fill negative area
axes[2,0].fill_between(econ_data['date'], econ_data['cumulative_gdp_impact'], 0, 
                       where=(econ_data['cumulative_gdp_impact'] < 0), 
                       color='red', alpha=0.3, label='GDP Loss')
axes[2,0].fill_between(econ_data['date'], econ_data['cumulative_gdp_impact'], 0, 
                       where=(econ_data['cumulative_gdp_impact'] > 0), 
                       color='green', alpha=0.3, label='GDP Gain')
axes[2,0].legend()

# Plot 6: Government Debt Level
axes[2,1].plot(econ_data['date'], econ_data['government_debt_level'], 
               color='orange', linewidth=3)
axes[2,1].set_ylabel('Additional Debt (% of GDP)')
axes[2,1].set_title('Government Debt Increase Due to Pandemic Response')
axes[2,1].grid(True, alpha=0.3)

# Add policy support overlay
ax_twin = axes[2,1].twinx()
ax_twin.bar(econ_data['date'], econ_data['policy_support_pct_gdp'] * 100, 
            alpha=0.3, color='green', width=20, label='Monthly Policy Support')
ax_twin.set_ylabel('Policy Support (% of GDP)', color='green')
ax_twin.tick_params(axis='y', labelcolor='green')
ax_twin.legend(loc='upper left')

plt.tight_layout()
plt.show()

# Calculate key statistics
pre_pandemic_gdp = econ_data[econ_data['phase'] == 'pre_pandemic']['gdp_growth_rate'].mean()
acute_phase_gdp = econ_data[econ_data['phase'] == 'acute_phase']['gdp_growth_rate'].mean()
peak_unemployment = econ_data['unemployment_rate'].max()
total_policy_support = econ_data['policy_support_pct_gdp'].sum()
max_debt_increase = econ_data['government_debt_level'].max()

print(f"\n📈 Key Economic Statistics:")
print(f"Pre-pandemic GDP growth: {pre_pandemic_gdp:.1f}% annually")
print(f"Acute phase GDP contraction: {acute_phase_gdp:.1f}% annually")
print(f"Peak unemployment rate: {peak_unemployment:.1f}%")
print(f"Total policy support: {total_policy_support:.1f}% of GDP")
print(f"Maximum debt increase: {max_debt_increase:.1f}% of GDP")
print(f"GDP impact at trough: {econ_data['cumulative_gdp_impact'].min():.1f}%")

## Part 3: Sectoral Analysis

In [ ]:
# Analyze differential impacts across economic sectors
sector_columns = ['services_growth', 'manufacturing_growth', 'agriculture_growth', 'technology_growth']
sector_names = ['Services', 'Manufacturing', 'Agriculture', 'Technology']

# Calculate average impact by phase for each sector
sector_impact_by_phase = econ_data.groupby('phase')[sector_columns].mean()

# Create sectoral analysis visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Sectoral growth rates over time
colors = ['blue', 'red', 'green', 'purple']
for i, (col, name, color) in enumerate(zip(sector_columns, sector_names, colors)):
    axes[0,0].plot(econ_data['date'], econ_data[col], 
                   color=color, linewidth=2, label=name)

axes[0,0].axhline(y=0, color='black', linestyle='--', alpha=0.7)
axes[0,0].set_ylabel('Growth Rate (%)')
axes[0,0].set_title('Sectoral Growth Rates Over Time')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Sectoral impact by phase (heatmap)
sns.heatmap(sector_impact_by_phase.T, annot=True, fmt='.1f', 
            cmap='RdYlGn', center=0, ax=axes[0,1])
axes[0,1].set_title('Average Sectoral Growth by Pandemic Phase')
axes[0,1].set_ylabel('Sector')
axes[0,1].set_xlabel('Pandemic Phase')

# Plot 3: Cumulative sectoral performance
for i, (col, name, color) in enumerate(zip(sector_columns, sector_names, colors)):
    cumulative_performance = ((1 + econ_data[col]/100).cumprod() - 1) * 100
    axes[1,0].plot(econ_data['date'], cumulative_performance, 
                   color=color, linewidth=2, label=name)

axes[1,0].axhline(y=0, color='black', linestyle='--', alpha=0.7)
axes[1,0].set_ylabel('Cumulative Performance (%)')
axes[1,0].set_title('Cumulative Sectoral Performance')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Sectoral volatility analysis
sector_volatility = econ_data[sector_columns].std()
sector_mean_growth = econ_data[sector_columns].mean()

axes[1,1].scatter(sector_volatility, sector_mean_growth, s=200, c=colors, alpha=0.7)

for i, (vol, growth, name) in enumerate(zip(sector_volatility, sector_mean_growth, sector_names)):
    axes[1,1].annotate(name, (vol, growth), xytext=(5, 5), 
                       textcoords='offset points', fontsize=10)

axes[1,1].set_xlabel('Volatility (Standard Deviation)')
axes[1,1].set_ylabel('Average Growth Rate (%)')
axes[1,1].set_title('Risk-Return Profile by Sector')
axes[1,1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate sectoral statistics
print(f"\n🏭 Sectoral Impact Analysis:")
for i, (col, name) in enumerate(zip(sector_columns, sector_names)):
    min_growth = econ_data[col].min()
    max_growth = econ_data[col].max()
    avg_growth = econ_data[col].mean()
    volatility = econ_data[col].std()
    
    print(f"{name}:")
    print(f"  Average growth: {avg_growth:.1f}%")
    print(f"  Minimum growth: {min_growth:.1f}%")
    print(f"  Maximum growth: {max_growth:.1f}%")
    print(f"  Volatility: {volatility:.1f}%")
    print()

# Identify most and least resilient sectors
acute_phase_impact = sector_impact_by_phase.loc['acute_phase']
most_resilient = acute_phase_impact.idxmax()
least_resilient = acute_phase_impact.idxmin()

print(f"📊 Resilience Analysis:")
print(f"Most resilient sector: {sector_names[sector_columns.index(most_resilient)]} ({acute_phase_impact[most_resilient]:.1f}% growth during acute phase)")
print(f"Least resilient sector: {sector_names[sector_columns.index(least_resilient)]} ({acute_phase_impact[least_resilient]:.1f}% growth during acute phase)")

## Part 4: Labor Market Analysis

In [ ]:
# Generate detailed labor market data
def generate_labor_market_data(econ_data):
    """
    Generate detailed labor market statistics
    """
    labor_data = econ_data.copy()
    
    # Different demographic groups experience different unemployment patterns
    for i, row in labor_data.iterrows():
        base_unemployment = row['unemployment_rate']
        phase = row['phase']
        
        # Age-based unemployment differences
        if phase in ['acute_phase', 'second_wave']:
            youth_multiplier = 2.0  # Young workers hit harder
            senior_multiplier = 0.8  # Many retire early
        else:
            youth_multiplier = 1.5
            senior_multiplier = 0.9
        
        labor_data.loc[i, 'youth_unemployment'] = min(25, base_unemployment * youth_multiplier)
        labor_data.loc[i, 'prime_age_unemployment'] = base_unemployment
        labor_data.loc[i, 'senior_unemployment'] = base_unemployment * senior_multiplier
        
        # Education-based differences
        if phase in ['acute_phase', 'second_wave']:
            # Service workers (often lower education) hit harder
            low_ed_multiplier = 2.5
            high_ed_multiplier = 0.6  # Many can work remotely
        else:
            low_ed_multiplier = 1.8
            high_ed_multiplier = 0.7
        
        labor_data.loc[i, 'low_education_unemployment'] = min(30, base_unemployment * low_ed_multiplier)
        labor_data.loc[i, 'high_education_unemployment'] = base_unemployment * high_ed_multiplier
        
        # Remote work capability
        if phase in ['acute_phase', 'partial_recovery', 'second_wave']:
            remote_work_pct = 35 + np.random.normal(0, 5)  # Peak remote work
        elif phase == 'recovery':
            remote_work_pct = 25 + np.random.normal(0, 3)  # Some permanent shift
        else:
            remote_work_pct = 5 + np.random.normal(0, 2)   # Pre-pandemic baseline
        
        labor_data.loc[i, 'remote_work_pct'] = max(0, remote_work_pct)
        
        # Job vacancies and labor shortages
        if phase == 'recovery':
            # Labor shortages emerge as economy recovers
            job_vacancy_rate = 8 + np.random.normal(0, 1)
        elif phase in ['acute_phase', 'second_wave']:
            job_vacancy_rate = 2 + np.random.normal(0, 0.5)
        else:
            job_vacancy_rate = 4 + np.random.normal(0, 0.8)
        
        labor_data.loc[i, 'job_vacancy_rate'] = max(0, job_vacancy_rate)
        
        # Labor force participation
        if phase in ['acute_phase', 'second_wave']:
            participation_change = -3.0  # People leave labor force
        elif phase == 'recovery':
            months_since = (row['date'] - datetime(2021, 6, 1)).days / 30
            participation_change = max(-1.0, -3.0 + 0.1 * months_since)
        else:
            participation_change = np.random.normal(0, 0.2)
        
        baseline_participation = 63.0  # Pre-pandemic rate
        labor_data.loc[i, 'labor_force_participation'] = baseline_participation + participation_change
    
    return labor_data

# Generate labor market data
labor_data = generate_labor_market_data(econ_data)

# Create labor market analysis visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Plot 1: Unemployment by age group
age_groups = ['youth_unemployment', 'prime_age_unemployment', 'senior_unemployment']
age_labels = ['Youth (16-24)', 'Prime Age (25-54)', 'Senior (55+)']
age_colors = ['red', 'blue', 'green']

for group, label, color in zip(age_groups, age_labels, age_colors):
    axes[0,0].plot(labor_data['date'], labor_data[group], 
                   color=color, linewidth=2, label=label)

axes[0,0].set_ylabel('Unemployment Rate (%)')
axes[0,0].set_title('Unemployment by Age Group')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Unemployment by education level
education_groups = ['low_education_unemployment', 'high_education_unemployment']
education_labels = ['Low Education', 'High Education']
education_colors = ['orange', 'purple']

for group, label, color in zip(education_groups, education_labels, education_colors):
    axes[0,1].plot(labor_data['date'], labor_data[group], 
                   color=color, linewidth=2, label=label)

axes[0,1].set_ylabel('Unemployment Rate (%)')
axes[0,1].set_title('Unemployment by Education Level')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Plot 3: Remote work adoption
axes[0,2].plot(labor_data['date'], labor_data['remote_work_pct'], 
               color='brown', linewidth=3)
axes[0,2].set_ylabel('Remote Work (%)')
axes[0,2].set_title('Remote Work Adoption')
axes[0,2].grid(True, alpha=0.3)

# Plot 4: Labor force participation
axes[1,0].plot(labor_data['date'], labor_data['labor_force_participation'], 
               color='darkgreen', linewidth=2)
axes[1,0].axhline(y=63.0, color='black', linestyle='--', alpha=0.7, label='Pre-pandemic')
axes[1,0].set_ylabel('Participation Rate (%)')
axes[1,0].set_title('Labor Force Participation Rate')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Plot 5: Job vacancy rate
axes[1,1].plot(labor_data['date'], labor_data['job_vacancy_rate'], 
               color='coral', linewidth=2)
axes[1,1].set_ylabel('Job Vacancy Rate (%)')
axes[1,1].set_title('Job Vacancy Rate')
axes[1,1].grid(True, alpha=0.3)

# Plot 6: Beveridge curve (unemployment vs job vacancies)
colors_by_phase = [phase_colors[phase] for phase in labor_data['phase']]
scatter = axes[1,2].scatter(labor_data['unemployment_rate'], labor_data['job_vacancy_rate'], 
                           c=colors_by_phase, alpha=0.7, s=30)

# Add arrows to show progression
for i in range(0, len(labor_data)-1, 6):  # Every 6 months
    x1, y1 = labor_data.iloc[i]['unemployment_rate'], labor_data.iloc[i]['job_vacancy_rate']
    x2, y2 = labor_data.iloc[i+1]['unemployment_rate'], labor_data.iloc[i+1]['job_vacancy_rate']
    axes[1,2].annotate('', xy=(x2, y2), xytext=(x1, y1),
                       arrowprops=dict(arrowstyle='->', alpha=0.5, lw=0.5))

axes[1,2].set_xlabel('Unemployment Rate (%)')
axes[1,2].set_ylabel('Job Vacancy Rate (%)')
axes[1,2].set_title('Beveridge Curve (Unemployment vs Job Vacancies)')
axes[1,2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate labor market statistics
print(f"\n👥 Labor Market Impact Analysis:")

# Peak unemployment by group
peak_youth = labor_data['youth_unemployment'].max()
peak_prime = labor_data['prime_age_unemployment'].max()
peak_senior = labor_data['senior_unemployment'].max()
peak_low_ed = labor_data['low_education_unemployment'].max()
peak_high_ed = labor_data['high_education_unemployment'].max()

print(f"Peak unemployment rates:")
print(f"  Youth (16-24): {peak_youth:.1f}%")
print(f"  Prime age (25-54): {peak_prime:.1f}%")
print(f"  Senior (55+): {peak_senior:.1f}%")
print(f"  Low education: {peak_low_ed:.1f}%")
print(f"  High education: {peak_high_ed:.1f}%")

# Remote work peak
peak_remote = labor_data['remote_work_pct'].max()
current_remote = labor_data['remote_work_pct'].iloc[-1]
print(f"\nRemote work:")
print(f"  Peak adoption: {peak_remote:.1f}%")
print(f"  Current level: {current_remote:.1f}%")

# Labor force participation impact
min_participation = labor_data['labor_force_participation'].min()
participation_loss = 63.0 - min_participation
print(f"\nLabor force participation:")
print(f"  Minimum rate: {min_participation:.1f}%")
print(f"  Maximum decline: {participation_loss:.1f} percentage points")

## Part 5: Policy Response Analysis

In [ ]:
# Analyze different policy responses and their economic effects
def simulate_policy_scenarios(base_data):
    """
    Simulate different policy response scenarios
    """
    scenarios = {}
    
    # Scenario 1: Actual policy response (baseline)
    scenarios['Actual Response'] = base_data.copy()
    
    # Scenario 2: No policy response
    no_policy = base_data.copy()
    no_policy['policy_support_pct_gdp'] = 0
    # Without policy support, recession would be deeper and longer
    for i, row in no_policy.iterrows():
        if row['phase'] in ['acute_phase', 'partial_recovery', 'second_wave']:
            # GDP impact would be 50% worse without policy support
            no_policy.loc[i, 'gdp_growth_rate'] = row['gdp_growth_rate'] * 1.5
            # Unemployment would be higher
            no_policy.loc[i, 'unemployment_rate'] = min(25, row['unemployment_rate'] * 1.4)
    
    scenarios['No Policy Response'] = no_policy
    
    # Scenario 3: Larger policy response
    large_policy = base_data.copy()
    large_policy['policy_support_pct_gdp'] = base_data['policy_support_pct_gdp'] * 2
    # Larger response would reduce economic damage but increase debt
    for i, row in large_policy.iterrows():
        if row['phase'] in ['acute_phase', 'partial_recovery', 'second_wave']:
            # GDP impact would be 30% better with larger response
            improvement = abs(row['gdp_growth_rate']) * 0.3
            large_policy.loc[i, 'gdp_growth_rate'] = row['gdp_growth_rate'] + improvement
            # Unemployment would be lower
            large_policy.loc[i, 'unemployment_rate'] = max(2, row['unemployment_rate'] * 0.8)
    
    scenarios['Large Policy Response'] = large_policy
    
    # Scenario 4: Health-focused response (stricter lockdowns, better health outcomes)
    health_focused = base_data.copy()
    # Stricter lockdowns initially, but faster recovery due to better health outcomes
    for i, row in health_focused.iterrows():
        if row['phase'] == 'acute_phase':
            # Deeper initial contraction
            health_focused.loc[i, 'gdp_growth_rate'] = row['gdp_growth_rate'] * 1.2
            health_focused.loc[i, 'unemployment_rate'] = min(20, row['unemployment_rate'] * 1.1)
        elif row['phase'] in ['partial_recovery', 'recovery']:
            # But faster recovery
            health_focused.loc[i, 'gdp_growth_rate'] = max(row['gdp_growth_rate'], row['gdp_growth_rate'] * 1.3)
            health_focused.loc[i, 'unemployment_rate'] = max(2, row['unemployment_rate'] * 0.9)
    
    scenarios['Health-Focused Response'] = health_focused
    
    return scenarios

# Generate policy scenarios
policy_scenarios = simulate_policy_scenarios(econ_data)

# Calculate cumulative impacts for each scenario
for name, data in policy_scenarios.items():
    data['cumulative_gdp_impact'] = ((1 + data['gdp_growth_rate']/100).cumprod() - 1) * 100
    data['government_debt_level'] = data['policy_support_pct_gdp'].cumsum() * 100

# Create policy comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

scenario_colors = {'Actual Response': 'blue', 'No Policy Response': 'red', 
                  'Large Policy Response': 'green', 'Health-Focused Response': 'purple'}

# Plot 1: GDP impact comparison
for name, data in policy_scenarios.items():
    axes[0,0].plot(data['date'], data['cumulative_gdp_impact'], 
                   color=scenario_colors[name], linewidth=2, label=name)

axes[0,0].axhline(y=0, color='black', linestyle='--', alpha=0.7)
axes[0,0].set_ylabel('Cumulative GDP Impact (%)')
axes[0,0].set_title('GDP Impact by Policy Scenario')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Unemployment comparison
for name, data in policy_scenarios.items():
    axes[0,1].plot(data['date'], data['unemployment_rate'], 
                   color=scenario_colors[name], linewidth=2, label=name)

axes[0,1].set_ylabel('Unemployment Rate (%)')
axes[0,1].set_title('Unemployment by Policy Scenario')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Plot 3: Government debt comparison
for name, data in policy_scenarios.items():
    axes[1,0].plot(data['date'], data['government_debt_level'], 
                   color=scenario_colors[name], linewidth=2, label=name)

axes[1,0].set_ylabel('Additional Government Debt (% of GDP)')
axes[1,0].set_title('Government Debt by Policy Scenario')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Policy effectiveness analysis
# Calculate final outcomes for each scenario
scenario_outcomes = []
for name, data in policy_scenarios.items():
    final_gdp_impact = data['cumulative_gdp_impact'].iloc[-1]
    max_unemployment = data['unemployment_rate'].max()
    total_debt = data['government_debt_level'].iloc[-1]
    avg_unemployment = data['unemployment_rate'].mean()
    
    scenario_outcomes.append({
        'Scenario': name,
        'Final GDP Impact': final_gdp_impact,
        'Peak Unemployment': max_unemployment,
        'Total Debt': total_debt,
        'Avg Unemployment': avg_unemployment
    })

outcomes_df = pd.DataFrame(scenario_outcomes)

# Create scatter plot of debt vs GDP impact
for i, row in outcomes_df.iterrows():
    color = scenario_colors[row['Scenario']]
    axes[1,1].scatter(row['Total Debt'], row['Final GDP Impact'], 
                     s=row['Peak Unemployment']*10, c=color, alpha=0.7, 
                     label=row['Scenario'])
    axes[1,1].annotate(row['Scenario'], 
                      (row['Total Debt'], row['Final GDP Impact']),
                      xytext=(5, 5), textcoords='offset points', fontsize=8)

axes[1,1].set_xlabel('Total Government Debt Increase (% of GDP)')
axes[1,1].set_ylabel('Final GDP Impact (%)')
axes[1,1].set_title('Policy Trade-offs: Debt vs Economic Outcome\n(Bubble size = Peak Unemployment)')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print policy comparison results
print(f"\n🏛️ Policy Scenario Comparison:")
print(outcomes_df.round(1).to_string(index=False))

# Calculate policy effectiveness metrics
print(f"\n📊 Policy Effectiveness Analysis:")
baseline = outcomes_df[outcomes_df['Scenario'] == 'No Policy Response'].iloc[0]
actual = outcomes_df[outcomes_df['Scenario'] == 'Actual Response'].iloc[0]

gdp_improvement = actual['Final GDP Impact'] - baseline['Final GDP Impact']
unemployment_reduction = baseline['Peak Unemployment'] - actual['Peak Unemployment']
debt_cost = actual['Total Debt']

print(f"Actual policy response vs no policy:")
print(f"  GDP improvement: {gdp_improvement:.1f} percentage points")
print(f"  Peak unemployment reduction: {unemployment_reduction:.1f} percentage points")
print(f"  Debt cost: {debt_cost:.1f}% of GDP")
print(f"  GDP improvement per debt point: {gdp_improvement/debt_cost:.2f}")

## Part 6: Economic Inequality Analysis

In [ ]:
# Analyze how the pandemic affected economic inequality
def generate_inequality_data(base_data):
    """
    Generate data on inequality impacts
    """
    inequality_data = base_data.copy()
    
    # Income quintiles
    quintiles = ['bottom_quintile', 'second_quintile', 'middle_quintile', 
                'fourth_quintile', 'top_quintile']
    
    for i, row in inequality_data.iterrows():
        phase = row['phase']
        base_gdp_growth = row['gdp_growth_rate']
        
        if phase in ['acute_phase', 'second_wave']:
            # Bottom quintiles hit much harder (service jobs, can't work from home)
            quintile_multipliers = [2.5, 2.0, 1.5, 1.0, 0.3]
        elif phase == 'partial_recovery':
            quintile_multipliers = [1.8, 1.4, 1.2, 1.0, 0.5]
        elif phase == 'recovery':
            # Asset price increases benefit top quintile
            quintile_multipliers = [1.2, 1.1, 1.0, 0.9, 0.2]
        else:  # pre_pandemic
            quintile_multipliers = [1.0, 1.0, 1.0, 1.0, 1.0]
        
        # Apply multipliers to create income growth by quintile
        for quintile, multiplier in zip(quintiles, quintile_multipliers):
            if base_gdp_growth < 0:  # Recession
                income_impact = base_gdp_growth * multiplier
            else:  # Growth
                # During growth, higher quintiles benefit more
                if quintile in ['top_quintile', 'fourth_quintile']:
                    income_impact = base_gdp_growth * (2 - multiplier)  # Inverse relationship
                else:
                    income_impact = base_gdp_growth * multiplier
            
            inequality_data.loc[i, f'{quintile}_income_growth'] = income_impact
        
        # Wealth effects (asset prices)
        stock_return = row['stock_market_return']
        
        # Wealth concentration: top quintile owns most financial assets
        wealth_shares = [0.02, 0.03, 0.08, 0.27, 0.60]  # Share of financial wealth
        
        for quintile, wealth_share in zip(quintiles, wealth_shares):
            wealth_effect = stock_return * wealth_share * 0.5  # 50% of wealth in stocks
            inequality_data.loc[i, f'{quintile}_wealth_effect'] = wealth_effect
        
        # Housing market effects
        if phase in ['recovery']:
            housing_return = 15 + np.random.normal(0, 3)  # Housing boom
        elif phase in ['acute_phase', 'second_wave']:
            housing_return = -5 + np.random.normal(0, 2)
        else:
            housing_return = 5 + np.random.normal(0, 2)
        
        # Homeownership rates vary by income
        homeownership_rates = [0.25, 0.45, 0.65, 0.80, 0.90]
        
        for quintile, ownership_rate in zip(quintiles, homeownership_rates):
            housing_effect = housing_return * ownership_rate * 0.3  # 30% of wealth in housing
            inequality_data.loc[i, f'{quintile}_housing_effect'] = housing_effect
    
    return inequality_data

# Generate inequality data
inequality_data = generate_inequality_data(econ_data)

# Calculate total wealth change by quintile
quintiles = ['bottom_quintile', 'second_quintile', 'middle_quintile', 
            'fourth_quintile', 'top_quintile']
quintile_labels = ['Bottom 20%', '2nd Quintile', 'Middle 20%', '4th Quintile', 'Top 20%']

for quintile in quintiles:
    income_col = f'{quintile}_income_growth'
    wealth_col = f'{quintile}_wealth_effect'
    housing_col = f'{quintile}_housing_effect'
    
    # Total effect combines income, wealth, and housing
    total_effect = (inequality_data[income_col] + 
                   inequality_data[wealth_col] + 
                   inequality_data[housing_col])
    
    inequality_data[f'{quintile}_total_effect'] = total_effect
    inequality_data[f'{quintile}_cumulative'] = ((1 + total_effect/100).cumprod() - 1) * 100

# Create inequality analysis visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

quintile_colors = ['red', 'orange', 'yellow', 'lightgreen', 'darkgreen']

# Plot 1: Income growth by quintile
for quintile, label, color in zip(quintiles, quintile_labels, quintile_colors):
    income_col = f'{quintile}_income_growth'
    axes[0,0].plot(inequality_data['date'], inequality_data[income_col], 
                   color=color, linewidth=2, label=label)

axes[0,0].axhline(y=0, color='black', linestyle='--', alpha=0.7)
axes[0,0].set_ylabel('Income Growth Rate (%)')
axes[0,0].set_title('Income Growth by Income Quintile')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Cumulative wealth change by quintile
for quintile, label, color in zip(quintiles, quintile_labels, quintile_colors):
    cumulative_col = f'{quintile}_cumulative'
    axes[0,1].plot(inequality_data['date'], inequality_data[cumulative_col], 
                   color=color, linewidth=2, label=label)

axes[0,1].axhline(y=0, color='black', linestyle='--', alpha=0.7)
axes[0,1].set_ylabel('Cumulative Wealth Change (%)')
axes[0,1].set_title('Cumulative Wealth Impact by Quintile')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Plot 3: Inequality measures over time
# Calculate Gini coefficient approximation
gini_approx = []
for i, row in inequality_data.iterrows():
    # Simplified Gini calculation based on quintile effects
    quintile_effects = [row[f'{q}_total_effect'] for q in quintiles]
    # Higher spread = higher inequality
    gini = np.std(quintile_effects) / np.mean(np.abs(quintile_effects)) if np.mean(np.abs(quintile_effects)) > 0 else 0
    gini_approx.append(gini)

inequality_data['gini_approximation'] = gini_approx

axes[1,0].plot(inequality_data['date'], inequality_data['gini_approximation'], 
               color='purple', linewidth=3)
axes[1,0].set_ylabel('Inequality Index')
axes[1,0].set_title('Economic Inequality Over Time')
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Final inequality comparison
final_effects = []
for quintile, label in zip(quintiles, quintile_labels):
    final_effect = inequality_data[f'{quintile}_cumulative'].iloc[-1]
    final_effects.append(final_effect)

bars = axes[1,1].bar(quintile_labels, final_effects, color=quintile_colors, alpha=0.7)
axes[1,1].axhline(y=0, color='black', linestyle='--', alpha=0.7)
axes[1,1].set_ylabel('Final Wealth Change (%)')
axes[1,1].set_title('Final Wealth Impact by Income Group')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, value in zip(bars, final_effects):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + (1 if height > 0 else -3),
                   f'{value:.1f}%', ha='center', va='bottom' if height > 0 else 'top',
                   fontweight='bold')

plt.tight_layout()
plt.show()

# Print inequality analysis results
print(f"\n⚖️ Economic Inequality Analysis:")
print(f"Final wealth change by income group:")
for label, effect in zip(quintile_labels, final_effects):
    print(f"  {label}: {effect:+.1f}%")

# Calculate inequality ratios
top_bottom_ratio = final_effects[-1] / final_effects[0] if final_effects[0] != 0 else float('inf')
print(f"\nInequality metrics:")
print(f"  Top-to-bottom quintile ratio: {top_bottom_ratio:.1f}")
print(f"  Spread (top - bottom): {final_effects[-1] - final_effects[0]:.1f} percentage points")

# Analyze drivers of inequality
print(f"\n📊 Drivers of Inequality:")
for quintile, label in zip(['bottom_quintile', 'top_quintile'], ['Bottom 20%', 'Top 20%']):
    income_effect = inequality_data[f'{quintile}_income_growth'].sum()
    wealth_effect = inequality_data[f'{quintile}_wealth_effect'].sum()
    housing_effect = inequality_data[f'{quintile}_housing_effect'].sum()
    
    print(f"{label}:")
    print(f"  Income effect: {income_effect:.1f}%")
    print(f"  Wealth effect: {wealth_effect:.1f}%")
    print(f"  Housing effect: {housing_effect:.1f}%")
    print()

## Part 7: Cost-Benefit Analysis of Public Health Interventions

In [ ]:
# Economic analysis of public health interventions
def calculate_intervention_costs_benefits():
    """
    Calculate economic costs and benefits of different public health interventions
    """
    interventions = {
        'School Closures': {
            'direct_cost_billion': 150,  # Lost education productivity
            'indirect_cost_billion': 300,  # Parents unable to work
            'health_benefit_billion': 50,  # Reduced transmission
            'duration_months': 6
        },
        'Business Lockdowns': {
            'direct_cost_billion': 500,  # Lost business revenue
            'indirect_cost_billion': 200,  # Supply chain effects
            'health_benefit_billion': 200,  # Major transmission reduction
            'duration_months': 3
        },
        'Mask Mandates': {
            'direct_cost_billion': 10,  # Cost of masks
            'indirect_cost_billion': 5,  # Compliance costs
            'health_benefit_billion': 100,  # Reduced transmission
            'duration_months': 18
        },
        'Travel Restrictions': {
            'direct_cost_billion': 300,  # Lost tourism/travel revenue
            'indirect_cost_billion': 100,  # Economic disruption
            'health_benefit_billion': 80,  # Reduced spread
            'duration_months': 12
        },
        'Contact Tracing': {
            'direct_cost_billion': 50,  # Program costs
            'indirect_cost_billion': 20,  # Quarantine costs
            'health_benefit_billion': 120,  # Early detection
            'duration_months': 24
        },
        'Vaccination Program': {
            'direct_cost_billion': 100,  # Vaccine development and distribution
            'indirect_cost_billion': 30,  # Administration costs
            'health_benefit_billion': 800,  # Massive health benefits
            'duration_months': 12
        }
    }
    
    # Calculate metrics for each intervention
    analysis = []
    for intervention, data in interventions.items():
        total_cost = data['direct_cost_billion'] + data['indirect_cost_billion']
        total_benefit = data['health_benefit_billion']
        net_benefit = total_benefit - total_cost
        benefit_cost_ratio = total_benefit / total_cost
        cost_per_month = total_cost / data['duration_months']
        
        analysis.append({
            'Intervention': intervention,
            'Total Cost ($B)': total_cost,
            'Health Benefit ($B)': total_benefit,
            'Net Benefit ($B)': net_benefit,
            'Benefit-Cost Ratio': benefit_cost_ratio,
            'Cost per Month ($B)': cost_per_month,
            'Duration (months)': data['duration_months']
        })
    
    return pd.DataFrame(analysis)

# Calculate intervention analysis
intervention_analysis = calculate_intervention_costs_benefits()

# Create cost-benefit analysis visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Cost vs Benefit scatter
scatter = axes[0,0].scatter(intervention_analysis['Total Cost ($B)'], 
                           intervention_analysis['Health Benefit ($B)'],
                           s=intervention_analysis['Duration (months)']*10,
                           c=intervention_analysis['Benefit-Cost Ratio'],
                           cmap='RdYlGn', alpha=0.7)

# Add diagonal line where benefit = cost
max_val = max(intervention_analysis['Total Cost ($B)'].max(), 
              intervention_analysis['Health Benefit ($B)'].max())
axes[0,0].plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Break-even')

# Annotate points
for i, row in intervention_analysis.iterrows():
    axes[0,0].annotate(row['Intervention'], 
                      (row['Total Cost ($B)'], row['Health Benefit ($B)']),
                      xytext=(5, 5), textcoords='offset points', fontsize=8)

axes[0,0].set_xlabel('Total Cost ($ Billions)')
axes[0,0].set_ylabel('Health Benefit ($ Billions)')
axes[0,0].set_title('Cost vs Benefit of Public Health Interventions\n(Bubble size = Duration)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[0,0], label='Benefit-Cost Ratio')

# Plot 2: Benefit-cost ratios
colors = ['red' if x < 1 else 'green' for x in intervention_analysis['Benefit-Cost Ratio']]
bars = axes[0,1].barh(intervention_analysis['Intervention'], 
                      intervention_analysis['Benefit-Cost Ratio'],
                      color=colors, alpha=0.7)

axes[0,1].axvline(x=1, color='black', linestyle='--', alpha=0.7, label='Break-even')
axes[0,1].set_xlabel('Benefit-Cost Ratio')
axes[0,1].set_title('Benefit-Cost Ratios by Intervention')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Add value labels
for bar, value in zip(bars, intervention_analysis['Benefit-Cost Ratio']):
    width = bar.get_width()
    axes[0,1].text(width + 0.1, bar.get_y() + bar.get_height()/2,
                   f'{value:.1f}', ha='left', va='center', fontweight='bold')

# Plot 3: Net benefits
colors = ['red' if x < 0 else 'green' for x in intervention_analysis['Net Benefit ($B)']]
bars = axes[1,0].bar(range(len(intervention_analysis)), 
                     intervention_analysis['Net Benefit ($B)'],
                     color=colors, alpha=0.7)

axes[1,0].axhline(y=0, color='black', linestyle='--', alpha=0.7)
axes[1,0].set_xlabel('Intervention')
axes[1,0].set_ylabel('Net Benefit ($ Billions)')
axes[1,0].set_title('Net Economic Benefit by Intervention')
axes[1,0].set_xticks(range(len(intervention_analysis)))
axes[1,0].set_xticklabels(intervention_analysis['Intervention'], rotation=45, ha='right')
axes[1,0].grid(True, alpha=0.3)

# Add value labels
for bar, value in zip(bars, intervention_analysis['Net Benefit ($B)']):
    height = bar.get_height()
    axes[1,0].text(bar.get_x() + bar.get_width()/2., height + (5 if height > 0 else -15),
                   f'${value:.0f}B', ha='center', va='bottom' if height > 0 else 'top',
                   fontweight='bold', fontsize=9)

# Plot 4: Cost efficiency (cost per month)
efficiency_order = intervention_analysis.sort_values('Cost per Month ($B)')
bars = axes[1,1].barh(efficiency_order['Intervention'], 
                      efficiency_order['Cost per Month ($B)'],
                      color='skyblue', alpha=0.7)

axes[1,1].set_xlabel('Cost per Month ($ Billions)')
axes[1,1].set_title('Cost Efficiency of Interventions')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print cost-benefit analysis results
print(f"\n💰 Cost-Benefit Analysis of Public Health Interventions:")
print(intervention_analysis.round(1).to_string(index=False))

# Identify best and worst interventions
best_ratio = intervention_analysis.loc[intervention_analysis['Benefit-Cost Ratio'].idxmax()]
best_net = intervention_analysis.loc[intervention_analysis['Net Benefit ($B)'].idxmax()]
most_efficient = intervention_analysis.loc[intervention_analysis['Cost per Month ($B)'].idxmin()]

print(f"\n🏆 Top Performing Interventions:")
print(f"Best benefit-cost ratio: {best_ratio['Intervention']} ({best_ratio['Benefit-Cost Ratio']:.1f})")
print(f"Highest net benefit: {best_net['Intervention']} (${best_net['Net Benefit ($B)']:.0f}B)")
print(f"Most cost-efficient: {most_efficient['Intervention']} (${most_efficient['Cost per Month ($B)']:.1f}B/month)")

# Calculate total portfolio effect
total_cost = intervention_analysis['Total Cost ($B)'].sum()
total_benefit = intervention_analysis['Health Benefit ($B)'].sum()
overall_ratio = total_benefit / total_cost

print(f"\n📊 Overall Portfolio Analysis:")
print(f"Total cost of all interventions: ${total_cost:.0f} billion")
print(f"Total health benefits: ${total_benefit:.0f} billion")
print(f"Overall benefit-cost ratio: {overall_ratio:.1f}")
print(f"Net benefit: ${total_benefit - total_cost:.0f} billion")

## Part 8: Exercises and Discussion Questions

### 🎯 Practice Exercises:

1. **Policy Scenario Analysis**: Design your own policy response scenario and compare its outcomes to the existing scenarios.

2. **Sectoral Deep Dive**: Choose one sector (e.g., healthcare, education, hospitality) and analyze its specific economic challenges during the pandemic.

3. **International Comparison**: How might these economic patterns differ across countries with different economic structures?

4. **Long-term Modeling**: Extend the analysis to model long-term economic effects (5-10 years post-pandemic).

### 🤔 Discussion Questions:

1. **Interdisciplinary Connections**: How would you integrate this economic analysis with:
   - Epidemiological models of disease spread and intervention effectiveness
   - Socio-behavioral analysis of compliance with economic restrictions
   - Built environment studies of workplace safety and productivity

2. **Policy Trade-offs**: How should policymakers balance:
   - Short-term economic costs vs. long-term health benefits
   - Economic efficiency vs. equity concerns
   - National vs. global economic impacts

3. **Measurement Challenges**: What are the limitations of using GDP and unemployment to measure pandemic impacts? What other metrics should be considered?

4. **Future Preparedness**: Based on this analysis, what economic policies and institutions should be in place to better handle future pandemics?

### 📚 Further Reading:

- Jordà, Ò., Singh, S. R., & Taylor, A. M. (2020). Longer-run economic consequences of pandemics. NBER Working Paper.
- Guerrieri, V., et al. (2020). Macroeconomic implications of COVID-19: Can negative supply shocks cause demand shortages? NBER Working Paper.
- Chetty, R., et al. (2020). The economic impacts of COVID-19: Evidence from a new public database. NBER Working Paper.

### 💡 Key Takeaways:

- **Pandemics create both supply and demand shocks** that interact in complex ways
- **Sectoral heterogeneity** means some industries are much more vulnerable than others
- **Labor market impacts** vary dramatically across demographic groups
- **Policy responses** can significantly mitigate economic damage but come with fiscal costs
- **Economic inequality** can be substantially worsened by pandemic impacts
- **Public health interventions** generally have positive benefit-cost ratios when health benefits are properly valued
- **Speed of policy response** matters as much as the magnitude
- **Economic recovery** may take years and may not return to pre-pandemic trends

---

*This notebook demonstrates the economic perspective essential for understanding the full cost of pandemics and the value of preparedness investments. Economists provide crucial insights for evaluating policy trade-offs, designing effective interventions, and ensuring that economic considerations are properly balanced with public health objectives in pandemic planning and response.*